In [80]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions.col
import org.apache.spark.util.SizeEstimator.estimate
import scala.collection.AbstractSeq
import scala.collection.mutable
import scala.collection.mutable.{ArrayBuffer, ListBuffer}
import spark.implicits._

import org.apache.spark.sql._
import org.apache.spark.sql.functions.col
import org.apache.spark.util.SizeEstimator.estimate
import scala.collection.AbstractSeq
import scala.collection.mutable
import scala.collection.mutable.{ArrayBuffer, ListBuffer}
import spark.implicits._


In [78]:
def TimeElapsing(benchmarkFunction: => Unit, message:Boolean = false)(times:Int = 1): Double = {
    if(message) println("Benchmark: IS Starting ...")
    val startTime = System.nanoTime()
    for (_ <- 0 until times)
      benchmarkFunction
    val endTime = System.nanoTime()
    val timeElapsed = (endTime - startTime).toDouble / times.toDouble
    if(message) println(s"Operation Took $timeElapsed ms on average")
    timeElapsed
}

TimeElapsing: (benchmarkFunction: => Unit, message: Boolean)(times: Int)Double


In [55]:
  def insertTime(x:AbstractSeq[Int], n:Int, m:Int):Double = x match {
    case x:ArrayBuffer[Int] => timeElapsing(x.updated(m,0))(n)
    case x:ListBuffer[Int] => timeElapsing(x.updated(m,0))(n)
    case x:mutable.MutableList[Int] => timeElapsing(x.updated(m,0))(n)
    case x:mutable.Queue[Int] => timeElapsing(x.updated(m,0))(n)
    case x:mutable.ArrayStack[Int] => timeElapsing(x.updated(m,0))(n)
    case _ => -1
  }

  def benchmarkSeq(x:AbstractSeq[Int], n:Int, m:Int): Map[String, Double] = {
    Map(
      "volume" -> estimate(x),
      "head" -> timeElapsing(x.head)(n),
      "tail" -> timeElapsing(x.tail)(n),
      "apply" -> timeElapsing(x.apply(m))(n),
      "update" -> timeElapsing(x.updated(m,0))(n),
      "prepend" -> timeElapsing(0+:x)(n),
      "append" -> timeElapsing(x:+0)(n),
      "insert" -> insertTime(x, n, m)
    )
  }

  def benchmarkString(x:String, n:Int, m:Int): Map[String, Double] = {
    Map(
      "volume" -> estimate(x),
      "head" -> timeElapsing(x.head)(n),
      "tail" -> timeElapsing(x.tail)(n),
      "apply" -> timeElapsing(x.apply(m))(n),
      "update" -> timeElapsing(x.updated(m,0))(n),
      "prepend" -> timeElapsing("0"+x)(n),
      "append" -> timeElapsing(x+"0")(n),
      "insert" -> -1
    )
  }

  def benchmarkStringBuilder(x:StringBuilder, n:Int, m:Int): Map[String, Double] = {
    Map(
      "volume" -> estimate(x),
      "head" -> timeElapsing(x.head)(n),
      "tail" -> timeElapsing(x.tail)(n),
      "apply" -> timeElapsing(x.apply(m))(n),
      "update" -> timeElapsing(x.updated(m,0))(n),
      "prepend" -> timeElapsing("0"+x)(n),
      "append" -> timeElapsing(x+"0")(n),
      "insert" -> timeElapsing(x.updated(m,0))(n)
    )
  }

  def benchmarkArray(x:Array[Int], n:Int, m:Int): Map[String, Double] =  { Map(
      "volume" -> estimate(x),
      "head" -> timeElapsing(x.head)(n),
      "tail" -> timeElapsing(x.tail)(n),
      "apply" -> timeElapsing(x.apply(m))(n),
      "update" -> timeElapsing(x.updated(m,0))(n),
      "prepend" -> timeElapsing(0+:x)(n),
      "append" -> timeElapsing(x:+0)(n),
      "insert" -> timeElapsing(x.updated(m,0))(n))
}
val obj = new Object()
  def benchmarkArrayBoxed(x:Array[Object], n:Int, m:Int): Map[String, Double] =  { Map(
      "volume" -> estimate(x),
      "head" -> timeElapsing(x.head)(n),
      "tail" -> timeElapsing(x.tail)(n),
      "apply" -> timeElapsing(x.apply(m))(n),
      "update" -> timeElapsing(x.updated(m,0))(n),
      "prepend" -> timeElapsing(obj+:x)(n),
      "append" -> timeElapsing(x:+obj)(n),
      "insert" -> timeElapsing(x.updated(m,0))(n))
}

<console>:53: warning: unreachable code
           case x:mutable.Queue[Int] => timeElapsing(x.updated(m,0))(n)
                                                                    ^
insertTime: (x: scala.collection.AbstractSeq[Int], n: Int, m: Int)Double
benchmarkSeq: (x: scala.collection.AbstractSeq[Int], n: Int, m: Int)Map[String,Double]
benchmarkString: (x: String, n: Int, m: Int)Map[String,Double]
benchmarkStringBuilder: (x: StringBuilder, n: Int, m: Int)Map[String,Double]
benchmarkArray: (x: Array[Int], n: Int, m: Int)Map[String,Double]
benchmarkArrayBoxed: (x: Array[Object], n: Int, m: Int)Map[String,Double]


In [6]:
val sizes = (0 to 3).map(x => math.pow(10, x).toInt)

sizes: scala.collection.immutable.IndexedSeq[Int] = Vector(1, 10, 100, 1000)


In [72]:
val stats = for(s <- sizes) yield {
val integers = 0 until s
List(
      ("Immutable_List", integers.toList),
      ("Immutable_Stream", integers.toStream),
      ("Immutable_Vector", integers.toVector),
      ("Immutable_Queue", scala.collection.immutable.Queue(integers: _*)),
      ("Immutable_Range", integers),
      ("Immutable_String", "1" * s),
      ("Mutable_ArrayBuffer", scala.collection.mutable.ArrayBuffer(integers: _*)),
      ("Mutable_ListBuffer", scala.collection.mutable.ListBuffer(integers: _*)),
      ("Mutable_StringBuilder", new scala.collection.mutable.StringBuilder("1" * s)),
      ("Mutable_MutableList", scala.collection.mutable.MutableList(integers: _*)),
      ("Mutable_Queue", scala.collection.mutable.Queue(integers: _*)),
      ("Mutable_ArraySeq", scala.collection.mutable.ArraySeq(integers: _*)),
      ("Mutable_ArrayStack", scala.collection.mutable.ArrayStack(integers: _*)),
      ("Mutable_Array", integers.toArray),
         ("Mutable_Boxed_Array",  {
       val boxedArray = new Array[Object](s)
       var i = 0
       while(i<s){boxedArray(i) = obj; i+=1}
       boxedArray
     })

).map(x=> x match{
    case (c, cl: AbstractSeq[Int]) =>  Map("size" -> s.toString, "collection" -> c) ++ benchmarkSeq(cl,100,s-1).map(x=> (x._1,x._2.toString))
    case (c, cl: Array[Object]) => Map("size" -> s.toString, "collection" -> c) ++ benchmarkArrayBoxed(cl,100,s-1).map(x=> (x._1,x._2.toString))    
    case (c, cl: Array[Int]) => Map("size" -> s.toString, "collection" -> c) ++ benchmarkArray(cl,100,s-1).map(x=> (x._1,x._2.toString))
    case (c, cl: String) => Map("size" -> s.toString, "collection" -> c) ++ benchmarkString(cl,100,s-1).map(x=> (x._1,x._2.toString))
    case (c, cl: StringBuilder) => Map("size" -> s.toString, "collection" -> c) ++ benchmarkStringBuilder(cl,100,s-1).map(x=> (x._1,x._2.toString))
})
}

<console>:83: warning: non-variable type argument Int in type pattern scala.collection.AbstractSeq[Int] is unchecked since it is eliminated by erasure
           case (c, cl: AbstractSeq[Int]) =>  Map("size" -> s.toString, "collection" -> c) ++ benchmarkSeq(cl,100,s-1).map(x=> (x._1,x._2.toString))
                        ^
<console>:87: warning: unreachable code
           case (c, cl: StringBuilder) => Map("size" -> s.toString, "collection" -> c) ++ benchmarkStringBuilder(cl,100,s-1).map(x=> (x._1,x._2.toString))
                                                                                       ^
<console>:82: warning: match may not be exhaustive.
It would fail on the following input: (_, _)
       ).map(x=> x match{
                 ^
obj: Object = java.lang.Object@64199285
s: Int = 1000
stats: scala.collection.immutable.IndexedSeq[List[scala.collection.immutable.Map[String,String]]] = Vector(List(Map(collection -> Immutable_List, prepend -> 19.82, size -> 1, insert -> -1.0, hea

In [75]:
  val colNames = stats(0).head.toList.sortBy(_._1).map(_._1)
    .zipWithIndex.map(x => col("value")(x._2).as(x._1))

  stats.flatten.map(x => x.toList.sortBy(_._1).map(_._2))
    .toDF.select(colNames:_*)
    .coalesce(1).write.option("header","true").mode("overwrite")
    .csv("./collection_size_benchmark.csv")

colNames: List[org.apache.spark.sql.Column] = List(value[0] AS `append`, value[1] AS `apply`, value[2] AS `collection`, value[3] AS `head`, value[4] AS `insert`, value[5] AS `prepend`, value[6] AS `size`, value[7] AS `tail`, value[8] AS `update`, value[9] AS `volume`)


In [1]:
val integers = 1 to 10

Intitializing Scala interpreter ...

Spark Web UI available at http://pc:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1642580478209)
SparkSession available as 'spark'


integers: scala.collection.immutable.Range.Inclusive = Range 1 to 10


In [6]:
scala.collection.mutable.HashMap(integers.zipWithIndex:_*)

res4: scala.collection.mutable.HashMap[Int,Int] = Map(8 -> 7, 2 -> 1, 5 -> 4, 4 -> 3, 7 -> 6, 10 -> 9, 1 -> 0, 9 -> 8, 3 -> 2, 6 -> 5)


In [7]:
scala.collection.mutable.HashSet(integers:_*)

res5: scala.collection.mutable.HashSet[Int] = Set(9, 1, 5, 2, 6, 3, 10, 7, 4, 8)


In [8]:
scala.collection.immutable.TreeSet(integers:_*)

res6: scala.collection.immutable.TreeSet[Int] = TreeSet(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


In [9]:
scala.collection.mutable.TreeSet(integers:_*)

res7: scala.collection.mutable.TreeSet[Int] = TreeSet(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


In [66]:
val integers = 0 until 10

val setData = List(
  ("Immutable_HashSet", integers.toSet),
  ("Immutable_TreeSet", scala.collection.immutable.TreeSet(integers:_*)),
  ("Immutable_BitSet", scala.collection.immutable.BitSet(integers:_*)),
  ("Mutable_HashSet", scala.collection.mutable.HashSet(integers:_*)),
  ("Mutable_BitSet", scala.collection.mutable.BitSet(integers:_*)),
  ("Mutable_TreeSet", scala.collection.mutable.TreeSet(integers:_*))
)

integers: scala.collection.immutable.Range = Range 0 until 10
mapData: List[(String, scala.collection.Map[Int,Int])] = List((Immutable_HashMap,Map(0 -> 0, 5 -> 5, 1 -> 1, 6 -> 6, 9 -> 9, 2 -> 2, 7 -> 7, 3 -> 3, 8 -> 8, 4 -> 4)), (Immutable_TreeMap,Map(0 -> 0, 1 -> 1, 2 -> 2, 3 -> 3, 4 -> 4, 5 -> 5, 6 -> 6, 7 -> 7, 8 -> 8, 9 -> 9)), (Immutable_ListMap,ListMap(0 -> 0, 1 -> 1, 2 -> 2, 3 -> 3, 4 -> 4, 5 -> 5, 6 -> 6, 7 -> 7, 8 -> 8, 9 -> 9)), (Mutable_HashMap,Map(8 -> 8, 2 -> 2, 5 -> 5, 4 -> 4, 7 -> 7, 1 -> 1, 9 -> 9, 3 -> 3, 6 -> 6, 0 -> 0)), (Mutable_WeakHashMap,Map(9 -> 9, 8 -> 8, 7 -> 7, 6 -> 6, 5 -> 5, 4 -> 4, 3 -> 3, 2 -> 2, 1 -> 1, 0 -> 0)))
setData: List[(String, scala.collection.Set[Int])] = List((Immutable_HashSet,Set(0, 5, 1, 6, 9, 2, 7, 3, 8, 4)), (Immutable_TreeSet,TreeSet(0, 1...


In [85]:
def benchmarkMap(x:scala.collection.Map[Int,Int], n:Int, m:Int): Map[String, Double] = {
Map(
  "volume" -> estimate(x),
  "lookup" -> TimeElapsing(x.get(m))(n),
  "add" -> TimeElapsing(x ++ Map((m,m)))(n),
  "remove" -> TimeElapsing(x-0)(n),
  "min" -> TimeElapsing(x.minBy(_._2)._1)(n)
)
}


benchmarkMap: (x: scala.collection.Map[Int,Int], n: Int, m: Int)Map[String,Double]


In [86]:
mapData.map(x => benchmarkMap(x._2,100,0))

res62: List[Map[String,Double]] = List(Map(lookup -> 8623.91, min -> 20361.98, remove -> 1905.93, add -> 4469.96, volume -> 800.0), Map(lookup -> 1403.39, min -> 8616.06, remove -> 8135.97, add -> 12646.31, volume -> 520.0), Map(lookup -> 2252.95, min -> 5972.79, remove -> 6155.64, add -> 19913.7, volume -> 416.0), Map(lookup -> 827.29, min -> 4900.1, remove -> 8279.22, add -> 19226.81, volume -> 520.0), Map(lookup -> 1441.47, min -> 14441.93, remove -> 23187.98, add -> 37944.69, volume -> 864.0))


In [49]:
a.map(x => x match {
    case x:scala.collection.immutable.Map[Int,Int] => "a"
    case _ => "b"
})

res41: List[String] = List(a, a)


In [38]:
integers.zipWithIndex.toMap

res34: scala.collection.immutable.Map[Int,Int] = Map(0 -> 0, 5 -> 5, 1 -> 1, 6 -> 6, 9 -> 9, 2 -> 2, 7 -> 7, 3 -> 3, 8 -> 8, 4 -> 4)


In [87]:
def benchmarkSet(x:scala.collection.Set[Int], n:Int, m:Int): Map[String, Double] = {
Map(
  "volume" -> estimate(x),
  "lookup" -> TimeElapsing(x.contains(m))(n),
  "add" -> TimeElapsing(x ++ Map((m,m)))(n),
  "remove" -> TimeElapsing(x-0)(n),
  "min" -> TimeElapsing(x.min)(n)
)
}


benchmarkSet: (x: scala.collection.Set[Int], n: Int, m: Int)Map[String,Double]


In [89]:
val s = 10
List(
      ("Immutable_HashSet", integers.toSet),
      ("Immutable_TreeSet", scala.collection.immutable.TreeSet(integers:_*)),
      ("Immutable_BitSet", scala.collection.immutable.BitSet(integers:_*)),
      ("Mutable_HashSet", scala.collection.mutable.HashSet(integers:_*)),
      ("Mutable_BitSet", scala.collection.mutable.BitSet(integers:_*)),
      ("Mutable_TreeSet", scala.collection.mutable.TreeSet(integers:_*))
    ).map(x => {
      Map("size" -> s.toString, "collection" -> x._1) ++ benchmarkSet(x._2, 100, s).map(x => (x._1, x._2.toString))
    })

s: Int = 10
res64: List[scala.collection.immutable.Map[String,String]] = List(Map(lookup -> 994.78, collection -> Immutable_HashSet, size -> 10, min -> 37298.86, remove -> 874.64, add -> 6935.41, volume -> 480.0), Map(lookup -> 1316.71, collection -> Immutable_TreeSet, size -> 10, min -> 2727.17, remove -> 5584.12, add -> 9209.83, volume -> 536.0), Map(lookup -> 413.83, collection -> Immutable_BitSet, size -> 10, min -> 23537.52, remove -> 1094.3, add -> 9511.25, volume -> 24.0), Map(lookup -> 362.25, collection -> Mutable_HashSet, size -> 10, min -> 6967.31, remove -> 9045.38, add -> 12348.32, volume -> 344.0), Map(lookup -> 501.97, collection -> Mutable_BitSet, size -> 10, min -> 22165.94, remove -> 4967.33, add -> 39198.21, volume -> 40.0), Map(lookup -> 1547.13, collection -> Mutabl...
